In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Previous version
class Pedro_Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    # iterative structure
    def backward(self):
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)

            if x.creator is not None:
                funcs.append(x.creator)


class Pedro_Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Pedro_Variable(y)
        output.set_creator(self)
        self.input = input
        self.output = output
        return output

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, gy):
        raise NotImplementedError()

class Pedro_Square(Pedro_Function):
    def forward(self, x):
        return x ** 2

    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx

class Pedro_Exp(Pedro_Function):
    def forward(self, x):
        return np.exp(x)

    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx


def numerical_diff(f, x, eps=1e-4):
    x0 = Pedro_Variable(x.data - eps)
    x1 = Pedro_Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) / (2 * eps)

In [3]:
# New version - update of functions
def square(x):
    f = Pedro_Square()
    return f(x)

def exp(x):
    f = Pedro_Exp()
    return f(x)


In [4]:
x = Pedro_Variable(np.array(0.5))
a = square(x)
b = exp(a)
y = square(b)

y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


In [5]:
class Pedro_Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)

            if x.creator is not None:
                funcs.append(x.creator)

In [6]:
x = Pedro_Variable(np.array(0.5))
a = square(x)
b = exp(a)
y = square(b)

y.backward()
print(x.grad)

3.297442541400256


In [7]:
class Pedro_Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError("Not supported!!")

        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)

            if x.creator is not None:
                funcs.append(x.creator)

In [8]:
x = Pedro_Variable(np.array(1.0))
print(x.data)

y = Pedro_Variable(None)
print(y.data)

z = Pedro_Variable(1.0)
print(z.data)

1.0
None


TypeError: Not supported!!

In [9]:
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

In [10]:
class Pedro_Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Pedro_Variable(as_array(y))
        output.set_creator(self)
        self.input = input
        self.output = output
        return output

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, gy):
        raise NotImplementedError()